## [安装python-can](https://python-can.readthedocs.io/en/master/installation.html)

In [1]:
import struct
import can
import time

In [2]:
MAIN_CAN_ID = 254 # 电脑上位机（主机） CAN ID
MOTOR_ID = 127 # 电机（从机） CAN ID

P_MIN = -12.5
P_MAX = 12.5
V_MIN = -30.0
V_MAX = 30.0

# 通信类型
class CmdModes:
    GET_DEVICE_ID = 0
    MOTOR_CONTROL = 1
    MOTOR_FEEDBACK = 2
    MOTOR_ENABLE = 3
    MOTOR_STOP = 4
    SET_MECHANICAL_ZERO = 6
    SET_MOTOR_CAN_ID = 7
    SINGLE_PARAM_READ = 17
    SINGLE_PARAM_WRITE = 18
    FAULT_FEEDBACK = 21


def float_to_uint(x, x_min, x_max, bits):
    span = x_max - x_min
    offset = x_min
    if x > x_max:
        x = x_max
    elif x < x_min:
        x = x_min
    return int(((x - offset) * ((1 << bits) - 1) / span))


def uint_to_float(x, x_min, x_max, bits):
    span = (1 << bits) - 1
    offset = x_max - x_min
    if x > span:
        x = span
    elif x < 0:
        x = 0
    return offset * x / span + x_min


def format_data(data=[], format="f f", type="decode"):
    # print(data)
    format_list = format.split()
    rdata = []
    if type == "decode":
        p = 0
        for f in format_list:
            s_f = []
            if f == "f":
                s_f = [4, "f"]
            elif f == "u16":
                s_f = [2, "H"]
            elif f == "s16":
                s_f = [2, "h"]
            elif f == "u32":
                s_f = [4, "I"]
            elif f == "s32":
                s_f = [4, "i"]
            elif f == "u8":
                s_f = [1, "B"]
            elif f == "s8":
                s_f = [1, "b"]
            ba = bytearray()
            if len(s_f) == 2:
                for i in range(s_f[0]):
                    ba.append(data[p])
                    p = p + 1
                rdata.append(struct.unpack(s_f[1], ba)[0])
            else:
                print("unkown format in format_data(): " + f)
                return []
        return rdata
    elif type == "encode" and len(format_list) == len(data):
        for i in range(len(format_list)):
            f = format_list[i]
            s_f = []
            if f == "f":
                s_f = [4, "f"]
            elif f == "u16":
                s_f = [2, "H"]
            elif f == "s16":
                s_f = [2, "h"]
            elif f == "u32":
                s_f = [4, "I"]
            elif f == "s32":
                s_f = [4, "i"]
            elif f == "u8":
                s_f = [1, "B"]
            elif f == "s8":
                s_f = [1, "b"]
            if f != "f":
                data[i] = int(data[i])
            if len(s_f) == 2:
                bs = struct.pack(s_f[1], data[i])
                for j in range(s_f[0]):
                    rdata.append(bs[j])
            else:
                print("unkown format in format_data(): " + f)
                return []
        if len(rdata) < 4:
            for i in range(4 - len(rdata)):
                rdata.append(0x00)
        return rdata

In [3]:
# 发生并接收一帧 CAN 信息
def send_receive_can_message(bus, cmd_mode, data2, motor_can_id, data1, timeout=0.2):
    """
    Send a CAN message and receive a response.

    Parameters:
    - bus: Initialized CAN bus object
    - cmd_mode: Command mode (bit28~bit24)
    - data2: Data area 2 (bit23~bit8)
    - motor_can_id: Motor target address (bit7~bit0)
    - data1: Data bytes to send
    - timeout: Timeout for sending the message (default is 0.2 seconds)

    Returns:
    A tuple containing the sent message and the received message, if any.
    """
    # Calculate the arbitration ID
    arbitration_id = (cmd_mode << 24) | (data2 << 8) | motor_can_id

    # Create CAN message
    message = can.Message(
        arbitration_id=arbitration_id, data=data1, is_extended_id=True,
    )

    # Send the CAN message
    try:
        bus.send(message, timeout=timeout)
    except can.CanError:
        print("Failed to send the message.")
        return None, None

    # Output details of the sent message
    print(f"Sent message with ID {hex(message.arbitration_id)}, data: 0x{(message.data.hex())}")

    # Receive a CAN message with a 1-second timeout
    received_msg = bus.recv(1.0)
    return message, received_msg

def parse_received_msg(received_msg):
    if received_msg is not None:
        print(
            f"Received message with ID {hex(received_msg.arbitration_id)}: {received_msg.data.hex()}"
        )
        pos = uint_to_float(
            (received_msg.data[0] << 8) + received_msg.data[1], P_MIN, P_MAX, 16
        )  # * RAD_DEG
        vel = uint_to_float(
            (received_msg.data[2] << 8) + received_msg.data[3], V_MIN, V_MAX, 16
        )  # * RAD_S_R_MIN
        print(f"pos:{pos:.2f} rad, vel:{vel:.2f} rad/s")
        return pos, vel
    else:
        print("No message received within the timeout period.")
        return None, None
    
# 清空CAN接收缓存，避免读到老数据
def clear_can_rx(timeout=0.1):
    # Clear the receiver buffer by reading all existing messages
    start_time = time.time()
    timeout = timeout  # seconds
    while time.time() - start_time < timeout:
        recv_msg = bus.recv(0)
        if recv_msg is not None:
            print(
                f"Cleared message with ID {hex(recv_msg.arbitration_id)}"
            )

In [4]:
# Connect to the CAN bus with 1 Mbit/s bitrate
bus = can.interface.Bus(bustype="pcan", channel="PCAN_USBBUS1", bitrate=1000000)

## Jog
单个参数写入（通信类型 18）

In [5]:
jog_vel = 5  # rad/s
uint_value = float_to_uint(jog_vel, V_MIN, V_MAX, 16)
jog_vel_bytes = format_data(data=[uint_value], format="u16", type="encode")[:2][::-1]

data1 = [0x05, 0x70, 0x00, 0x00, 0x07, 0x01] + jog_vel_bytes
clear_can_rx()
sent_msg, received_msg = send_receive_can_message(
    bus=bus, cmd_mode=CmdModes.SINGLE_PARAM_WRITE, data2=MAIN_CAN_ID, motor_can_id=MOTOR_ID, data1=data1
)
parse_received_msg(received_msg)

Sent message with ID 0x1200fe7f, data: 0x0570000007019554
Received message with ID 0x2807ffe: ffff7ffb7fff0196
pos:12.50 rad, vel:-0.00 rad/s


(12.5, -0.004119935912108019)

## 停止
单个参数写入（通信类型 18）

In [6]:
data1 = [0x05, 0x70, 0x00, 0x00, 0x07, 0x00, 0x7F, 0xFF]
clear_can_rx()
sent_msg, received_msg = send_receive_can_message(
    bus=bus, cmd_mode=CmdModes.SINGLE_PARAM_WRITE, data2=MAIN_CAN_ID, motor_can_id=MOTOR_ID, data1=data1
)
parse_received_msg(received_msg)

Sent message with ID 0x1200fe7f, data: 0x0570000007007fff
Received message with ID 0x2007ffe: ffff953c7fff0196
pos:12.50 rad, vel:4.98 rad/s


(12.5, 4.977340352483402)

## 发送电机模式参数写入命令（通信类型 18）

In [7]:
def write_single_param(index, value, format="u32"):
    encoded_data = format_data(data=[value], format=format, type="encode")
    data_bytes = struct.pack("<I", index) + bytes(encoded_data)

    clear_can_rx()  # 空CAN接收缓存，避免读到老数据

    sent_msg, received_msg = send_receive_can_message(
        bus=bus,
        cmd_mode=CmdModes.SINGLE_PARAM_WRITE,
        data2=MAIN_CAN_ID,
        motor_can_id=MOTOR_ID,
        data1=data_bytes,
    )
    parse_received_msg(received_msg)
    return received_msg

## 设置电机机械零位（通信类型6）
会把当前电机位置设为机械零位

In [9]:
def set_0_pos():
    clear_can_rx() # 空CAN接收缓存，避免读到老数据

    sent_msg, received_msg = send_receive_can_message(
        bus=bus,
        cmd_mode=CmdModes.SET_MECHANICAL_ZERO,
        data2=MAIN_CAN_ID,
        motor_can_id=MOTOR_ID,
        data1=[1],
    )
    parse_received_msg(received_msg)

## 电机使能运行 （通信类型 3）

In [13]:
def enable():
    clear_can_rx(0) # 空CAN接收缓存，避免读到老数据

    sent_msg, received_msg = send_receive_can_message(
        bus=bus,
        cmd_mode=CmdModes.MOTOR_ENABLE,
        data2=MAIN_CAN_ID,
        motor_can_id=MOTOR_ID,
        data1=[],
    )
    parse_received_msg(received_msg)

## 电机停止运行 （通信类型 4）

In [15]:
def disable():
    clear_can_rx(0)  # 空CAN接收缓存，避免读到老数据

    sent_msg, received_msg = send_receive_can_message(
        bus=bus,
        cmd_mode=CmdModes.MOTOR_STOP,
        data2=MAIN_CAN_ID,
        motor_can_id=MOTOR_ID,
        data1=[0, 0, 0, 0, 0, 0, 0, 0],
    )
    parse_received_msg(received_msg)

## 读取速度和位置
- 发送参数：单个参数写⼊（通信类型18），写入地址0x7018（电流限制）值为 27A
- 读取返回值：电机反馈数据（通信类型2），Byte0,1: 当前⻆度，Byte2,3:当前⻆速度

In [8]:
write_single_param(index=0x7018, value=27, format='f')

Sent message with ID 0x1200fe7f, data: 0x187000000000d841
Received message with ID 0x2007ffe: ffff80417fff0196
pos:12.50 rad, vel:0.06 rad/s


can.Message(timestamp=1694338120.242703, arbitration_id=0x2007ffe, is_extended_id=True, dlc=8, data=[0xff, 0xff, 0x80, 0x41, 0x7f, 0xff, 0x1, 0x96])

# 位置模式
## 发送电机模式参数写入命令（通信类型 18）
设置 `runmode` 参数为 1
- index(Byte0~1): `run_mode`，0x7005
- value(Byte4~7): 1(位置模式)

In [11]:
write_single_param(index=0x7005, value=1, format='u32')

Sent message with ID 0x1200fe7f, data: 0x0570000001000000
Received message with ID 0x2007ffe: 7fff80317fff0196
pos:-0.00 rad, vel:0.05 rad/s


can.Message(timestamp=1694338122.771685, arbitration_id=0x2007ffe, is_extended_id=True, dlc=8, data=[0x7f, 0xff, 0x80, 0x31, 0x7f, 0xff, 0x1, 0x96])

In [14]:
enable()

Sent message with ID 0x300fe7f, data: 0x
Received message with ID 0x2807ffe: 7fff80017fff0196
pos:-0.00 rad, vel:0.00 rad/s


## 最大速度：发送电机模式参数写入命令（通信类型 18）
设置 `limit_spd` 参数为预设最大速度指令
- index(Byte0~1): `limit_spd`, 0x7017
- value(Byte4~7): `float` [0,30]rad/s

In [16]:
write_single_param(index=0x7017, value=10, format='f')

Sent message with ID 0x1200fe7f, data: 0x1770000000002041
Received message with ID 0x2807ffe: 7fff801f80220196
pos:-0.00 rad, vel:0.03 rad/s


can.Message(timestamp=1694338138.7050588, arbitration_id=0x2807ffe, is_extended_id=True, dlc=8, data=[0x7f, 0xff, 0x80, 0x1f, 0x80, 0x22, 0x1, 0x96])

## 目标位置：发送电机模式参数写入命令（通信类型 18）
设置 `loc_ref` 参数为预设位置指令
- index(Byte0~1): `loc_ref`, 0x7016
- value(Byte4~7): `float` rad

In [17]:
write_single_param(index=0x7016, value=0, format='f')

Sent message with ID 0x1200fe7f, data: 0x1670000000000000
Received message with ID 0x2807ffe: 7ffe7fbf7f340196
pos:-0.00 rad, vel:-0.06 rad/s


can.Message(timestamp=1694338140.200372, arbitration_id=0x2807ffe, is_extended_id=True, dlc=8, data=[0x7f, 0xfe, 0x7f, 0xbf, 0x7f, 0x34, 0x1, 0x96])

In [18]:
def set_motor_position_control(limit_spd, loc_ref):
    # 设置电机最大速度
    write_single_param(index=0x7017, value=limit_spd, format="f")
    # 设置电机目标位置
    write_single_param(index=0x7016, value=loc_ref, format="f")

In [21]:
# 定义音符到电机参数的映射，为后面的音符分配更高的转速值
note_to_speed = {"C": 6.6, "D": 6.8, "E": 7.3, "F": 8, "G": 9, "A": 10, "B": 13}

# 定义小星星的音符和时长
# 完整定义小星星的音符和时长
notes = ["C", "C", "G", "G", "A", "A", "G",
         "F", "F", "E", "E", "D", "D", "C",
         "G", "G", "F", "F", "E", "E", "D",
         "G", "G", "F", "F", "E", "E", "D",
         "C", "C", "G", "G", "A", "A", "G",
         "F", "F", "E", "E", "D", "D", "C"]

durations = [1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2]  # 以四分音符为单位

x_speed = 1.5 # 速度倍数
x_duration = 0.4 # 间隔倍数，如果小于1，就是减小时间间隔


# 初始化电机的当前位置和方向标志
current_loc_ref = 0
direction_flag = 1  # 1表示正方向，-1表示反方向

disable() #
set_motor_position_control(limit_spd=10, loc_ref=0)
set_0_pos()  # 当前0位
enable()

# 播放小星星
for note, duration in zip(notes, durations):
    duration = duration * x_duration
    
    # 计算电机的转速和目标位置
    limit_spd = note_to_speed[note] * x_speed
    loc_ref = current_loc_ref + (direction_flag * limit_spd * duration * 1.5)

    # 使用电机参数设置函数
    set_motor_position_control(limit_spd=limit_spd, loc_ref=loc_ref)

    # 更新电机的当前位置
    current_loc_ref = loc_ref

    # 反转方向标志
#     direction_flag *= -1

    # 休眠一段时间以模拟音符的时长（这里假设一个四分音符的时长是1秒）
    time.sleep(duration * x_speed)
disable()

Sent message with ID 0x400fe7f, data: 0x0000000000000000
Received message with ID 0x2007ffe: ffff800b7fff0189
pos:12.50 rad, vel:0.01 rad/s
Sent message with ID 0x1200fe7f, data: 0x1770000000002041
Received message with ID 0x2007ffe: ffff7ffe7fff0189
pos:12.50 rad, vel:-0.00 rad/s
Sent message with ID 0x1200fe7f, data: 0x1670000000000000
Received message with ID 0x2007ffe: ffff80007fff0189
pos:12.50 rad, vel:0.00 rad/s
Sent message with ID 0x600fe7f, data: 0x01
Received message with ID 0x2007ffe: 7fff7f227fff0189
pos:-0.00 rad, vel:-0.20 rad/s
Sent message with ID 0x300fe7f, data: 0x
Received message with ID 0x2807ffe: 7ffe7fb27fff0189
pos:-0.00 rad, vel:-0.07 rad/s
Sent message with ID 0x1200fe7f, data: 0x1770000066661e41
Received message with ID 0x2807ffe: 7ffd80267fff0189
pos:-0.00 rad, vel:0.04 rad/s
Sent message with ID 0x1200fe7f, data: 0x167000007b14be40
Received message with ID 0x2807ffe: 7ffc80547fff0189
pos:-0.00 rad, vel:0.08 rad/s
Sent message with ID 0x1200fe7f, data: 0x17

Sent message with ID 0x1200fe7f, data: 0x1770000033332341
Received message with ID 0x2807ffe: ffff7e187fff0182
pos:12.50 rad, vel:-0.45 rad/s
Sent message with ID 0x1200fe7f, data: 0x1670000066e66143
Received message with ID 0x2807ffe: ffff7fe87fff0182
pos:12.50 rad, vel:-0.02 rad/s
Sent message with ID 0x1200fe7f, data: 0x1770000066661e41
Received message with ID 0x2807ffe: ffff7e5b7fff0182
pos:12.50 rad, vel:-0.38 rad/s
Sent message with ID 0x1200fe7f, data: 0x167000000ad76743
Received message with ID 0x2807ffe: ffff7fab7ea10182
pos:12.50 rad, vel:-0.08 rad/s
Sent message with ID 0x1200fe7f, data: 0x1770000066661e41
Received message with ID 0x2807ffe: ffff7e1f7fce0182
pos:12.50 rad, vel:-0.44 rad/s
Sent message with ID 0x1200fe7f, data: 0x16700000aec76d43
Received message with ID 0x2807ffe: ffff802b7fff0182
pos:12.50 rad, vel:0.04 rad/s
Sent message with ID 0x1200fe7f, data: 0x1770000000005841
Received message with ID 0x2807ffe: ffff7e277fff0182
pos:12.50 rad, vel:-0.43 rad/s
Sent me